In [6]:
import networkx as nx
from networkx import ego_graph

import torch.optim as optim
import argparse
import numpy as np
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv, GATConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

#from logger import Logger
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import WebKB
from torch_geometric.loader import DataLoader

In [2]:
dataset = WebKB(root='/tmp/Texas', name='Texas',transform=T.ToSparseTensor())
data = dataset[0]
#data.adj_t = data.adj_t.to_symmetric()
#data.adj_t = data.adj_t.to_symmetric()
print(data)
#split_idx = dataset.get_idx_split()
#train_idx = split_idx['train'].to(device)

Data(x=[183, 1703], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=325])


In [ ]:
train_index = np.where(data.train_mask)[0]
print(len(train_index))
valid_index = np.where(data.val_mask)[0]
print(len(valid_index))
test_index = np.where(data.test_mask)[0]
print(len(test_index))

# GAT

In [3]:
import torch


class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

In [13]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout,heads):
        super(GAT, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(GATConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GATConv(hidden_channels, out_channels))

        self.dropout = dropout
        self.heads=heads

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            #x=F.softmax(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)


def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    #print(len(out))
    #print(data.y.squeeze(1)[train_idx])
    loss = F.nll_loss(out, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total

@torch.no_grad()
def test(model, data, train_idx,valid_idx,test_idx):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [14]:
idx=[data.train_mask[i][0] for i in range(183)]
train_index = np.where(idx)[0]
print(train_index)

[  0   2   4   8  11  13  15  19  21  26  32  35  37  39  41  44  45  46
  50  53  58  59  60  61  63  65  66  70  71  72  73  74  75  76  77  78
  83  85  86  88  89  92  94  95  96 101 103 104 105 107 108 110 113 115
 116 119 120 121 123 124 125 126 130 131 133 135 137 139 140 141 142 144
 145 146 148 152 153 156 161 168 170 171 173 177 179 181 182]


In [23]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 8, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.6, 'epochs': 50, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01}

    args = objectview(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    dataset = WebKB(root='/tmp/Texas', name='Texas',transform=T.ToSparseTensor())
    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    model = GAT(data.num_features, args.hidden_channels, dataset.num_classes, args.num_layers,
                    args.dropout,args.heads)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.9275, Train: 28.74%, Valid: 8.47% Test: 10.81%
Run: 01, Epoch: 02, Loss: 1.6475, Train: 48.28%, Valid: 40.68% Test: 43.24%
Run: 01, Epoch: 03, Loss: 1.4614, Train: 41.38%, Valid: 13.56% Test: 32.43%
Run: 01, Epoch: 04, Loss: 1.3817, Train: 49.43%, Valid: 35.59% Test: 37.84%
Run: 01, Epoch: 05, Loss: 1.2988, Train: 59.77%, Valid: 44.07% Test: 45.95%
Run: 01, Epoch: 06, Loss: 1.2432, Train: 64.37%, Valid: 47.46% Test: 45.95%
Run: 01, Epoch: 07, Loss: 1.1850, Train: 66.67%, Valid: 50.85% Test: 48.65%
Run: 01, Epoch: 08, Loss: 1.1521, Train: 67.82%, Valid: 55.93% Test: 51.35%
Run: 01, Epoch: 09, Loss: 1.0468, Train: 68.97%, Valid: 55.93% Test: 51.35%
Run: 01, Epoch: 10, Loss: 1.0178, Train: 68.97%, Valid: 55.93% Test: 51.35%
Run: 01, Epoch: 11, Loss: 0.9633, Train: 68.97%, Valid: 55.93% Test: 54.05%
Run: 01, Epoch: 12, Loss: 0.9886, Train: 71.26%, Valid: 55.93% Test: 54.05%
Run: 01, Epoch: 13, Loss: 0.9261, Train: 72.41%, Valid: 57.63% Test: 54.05%
Run: 01, Epoc

Run: 04, Epoch: 28, Loss: 0.7579, Train: 78.16%, Valid: 61.02% Test: 62.16%
Run: 04, Epoch: 29, Loss: 0.6343, Train: 79.31%, Valid: 57.63% Test: 59.46%
Run: 04, Epoch: 30, Loss: 0.6579, Train: 78.16%, Valid: 57.63% Test: 56.76%
Run: 04, Epoch: 31, Loss: 0.6415, Train: 81.61%, Valid: 57.63% Test: 56.76%
Run: 04, Epoch: 32, Loss: 0.7373, Train: 81.61%, Valid: 54.24% Test: 56.76%
Run: 04, Epoch: 33, Loss: 0.5981, Train: 81.61%, Valid: 55.93% Test: 59.46%
Run: 04, Epoch: 34, Loss: 0.6285, Train: 81.61%, Valid: 55.93% Test: 59.46%
Run: 04, Epoch: 35, Loss: 0.6291, Train: 81.61%, Valid: 55.93% Test: 59.46%
Run: 04, Epoch: 36, Loss: 0.5708, Train: 82.76%, Valid: 55.93% Test: 59.46%
Run: 04, Epoch: 37, Loss: 0.6089, Train: 85.06%, Valid: 55.93% Test: 59.46%
Run: 04, Epoch: 38, Loss: 0.5500, Train: 85.06%, Valid: 54.24% Test: 59.46%
Run: 04, Epoch: 39, Loss: 0.6330, Train: 85.06%, Valid: 54.24% Test: 54.05%
Run: 04, Epoch: 40, Loss: 0.5771, Train: 86.21%, Valid: 54.24% Test: 54.05%
Run: 04, Epo

Run: 08, Epoch: 10, Loss: 0.9502, Train: 42.53%, Valid: 18.64% Test: 16.22%
Run: 08, Epoch: 11, Loss: 1.1204, Train: 48.28%, Valid: 20.34% Test: 18.92%
Run: 08, Epoch: 12, Loss: 0.9370, Train: 51.72%, Valid: 16.95% Test: 18.92%
Run: 08, Epoch: 13, Loss: 1.0675, Train: 52.87%, Valid: 18.64% Test: 16.22%
Run: 08, Epoch: 14, Loss: 1.1456, Train: 59.77%, Valid: 35.59% Test: 21.62%
Run: 08, Epoch: 15, Loss: 1.0544, Train: 63.22%, Valid: 37.29% Test: 35.14%
Run: 08, Epoch: 16, Loss: 0.8461, Train: 65.52%, Valid: 37.29% Test: 35.14%
Run: 08, Epoch: 17, Loss: 0.8259, Train: 66.67%, Valid: 42.37% Test: 32.43%
Run: 08, Epoch: 18, Loss: 0.9464, Train: 68.97%, Valid: 44.07% Test: 35.14%
Run: 08, Epoch: 19, Loss: 0.8533, Train: 70.11%, Valid: 44.07% Test: 40.54%
Run: 08, Epoch: 20, Loss: 0.8210, Train: 72.41%, Valid: 50.85% Test: 40.54%
Run: 08, Epoch: 21, Loss: 1.1380, Train: 77.01%, Valid: 50.85% Test: 45.95%
Run: 08, Epoch: 22, Loss: 0.8759, Train: 75.86%, Valid: 52.54% Test: 45.95%
Run: 08, Epo

# WISE-Embeddings

In [24]:
dataset = WebKB(root='/tmp/Texas', name='Texas',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[183, 1703], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=325])


In [25]:
import pandas as pd
Domain_Fec=pd.DataFrame(data.x.numpy())
label=pd.DataFrame(data.y.numpy(),columns =['class'])
Data=pd.concat([Domain_Fec,label], axis=1)
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,1694,1695,1696,1697,1698,1699,1700,1701,1702,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [26]:
Number_nodes=len(data.y)
fe_len=len(data.x[0])
catagories=Data['class'].to_numpy()
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class'], axis=1) for cls in range(max(catagories) + 1)}
basis = [[max(df[i]) for i in range(len(df.columns))] for df in data_by_class.values()]
sel_basis = [[int(list(df[i].to_numpy()).count(1) >= int(len(df[i].index)*0.1)) 
              for i in range(len(df.columns))]
             for df in data_by_class.values()]
feature_names = [ii for ii in range(fe_len)]

In [ ]:
#It takes long time
Fec=[]
for i in range(23):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=np.zeros(7)
    for j in range(1433):
        for i in range(max(catagories)+1):
            if f[j]==1 and basis[i][j]==1:
                count[i]=count[i]+1;

    for i in range(max(catagories)+1):
        vec.append(count[i])
    f.clear()
    Fec.append(vec)
print(Fec)

In [27]:
Fec=[]
for i in range(Number_nodes):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    for j in range(fe_len):
        if f[j]==1 and basis[0][j]==1:
            count=count+1;
        if f[j]==1 and basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and basis[4][j]==1:
            count4=count4+1;
    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)
    #print(f)
    f.clear()
    Fec.append(vec)

In [ ]:
print(Fec)

In [28]:
SFec=[]
for i in range(Number_nodes):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    for j in range(fe_len):
        if f[j]==1 and sel_basis[0][j]==1:
            count=count+1;
        if f[j]==1 and sel_basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and sel_basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and sel_basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and sel_basis[4][j]==1:
            count4=count4+1;
    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)
    #print(f)
    f.clear()
    SFec.append(Svec)

In [ ]:
print(SFec)

In [29]:
Inc_fe=torch.tensor(Fec)
sel_fe=torch.tensor(SFec)
CC_domain=torch.cat((Inc_fe, sel_fe), 1).float()
print(CC_domain)
CC_domain.type()

tensor([[ 39.,  20.,  40.,  ...,  40.,  30.,  32.],
        [231.,  25., 155.,  ..., 155.,  64.,  92.],
        [ 25.,  20.,  26.,  ...,  26.,  23.,  23.],
        ...,
        [ 77.,  32.,  73.,  ...,  73.,  49.,  67.],
        [ 61.,  28.,  56.,  ...,  56.,  54.,  51.],
        [ 41.,  23.,  37.,  ...,  37.,  36.,  32.]])


'torch.FloatTensor'

# W-GAT

In [32]:
data.x=CC_domain
print(data)

Data(x=[183, 10], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=574])


In [34]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 8, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.6, 'epochs': 50, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = WebKB(root='/tmp/Texas', name='Texas',transform=T.ToSparseTensor())
    #data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    #idx_train=[data.train_mask[i][0] for i in range(len(data.y))]
    #train_idx = np.where(idx_train)[0]
    #idx_val=[data.val_mask[i][0] for i in range(len(data.y))]
    #valid_idx = np.where(idx_val)[0]
    #idx_test=[data.test_mask[i][0] for i in range(len(data.y))]
    #test_idx = np.where(idx_test)[0]
    
    model = GAT(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout,args.heads)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 2.8866, Train: 20.69%, Valid: 11.86% Test: 16.22%
Run: 01, Epoch: 02, Loss: 3.1209, Train: 22.99%, Valid: 8.47% Test: 13.51%
Run: 01, Epoch: 03, Loss: 3.1153, Train: 22.99%, Valid: 8.47% Test: 13.51%
Run: 01, Epoch: 04, Loss: 1.8115, Train: 25.29%, Valid: 10.17% Test: 13.51%
Run: 01, Epoch: 05, Loss: 1.8254, Train: 26.44%, Valid: 10.17% Test: 13.51%
Run: 01, Epoch: 06, Loss: 1.2650, Train: 29.89%, Valid: 15.25% Test: 27.03%
Run: 01, Epoch: 07, Loss: 1.5370, Train: 36.78%, Valid: 33.90% Test: 27.03%
Run: 01, Epoch: 08, Loss: 1.4210, Train: 37.93%, Valid: 30.51% Test: 29.73%
Run: 01, Epoch: 09, Loss: 1.4059, Train: 33.33%, Valid: 16.95% Test: 29.73%
Run: 01, Epoch: 10, Loss: 1.3600, Train: 34.48%, Valid: 16.95% Test: 29.73%
Run: 01, Epoch: 11, Loss: 1.2994, Train: 41.38%, Valid: 28.81% Test: 29.73%
Run: 01, Epoch: 12, Loss: 1.2972, Train: 43.68%, Valid: 33.90% Test: 35.14%
Run: 01, Epoch: 13, Loss: 1.3159, Train: 47.13%, Valid: 37.29% Test: 54.05%
Run: 01, Epoch

Run: 04, Epoch: 37, Loss: 1.2021, Train: 56.32%, Valid: 54.24% Test: 59.46%
Run: 04, Epoch: 38, Loss: 1.2859, Train: 57.47%, Valid: 54.24% Test: 59.46%
Run: 04, Epoch: 39, Loss: 1.1942, Train: 57.47%, Valid: 54.24% Test: 59.46%
Run: 04, Epoch: 40, Loss: 1.1550, Train: 58.62%, Valid: 54.24% Test: 59.46%
Run: 04, Epoch: 41, Loss: 1.2884, Train: 58.62%, Valid: 54.24% Test: 59.46%
Run: 04, Epoch: 42, Loss: 1.2000, Train: 58.62%, Valid: 52.54% Test: 62.16%
Run: 04, Epoch: 43, Loss: 1.1026, Train: 57.47%, Valid: 52.54% Test: 62.16%
Run: 04, Epoch: 44, Loss: 1.2202, Train: 57.47%, Valid: 54.24% Test: 64.86%
Run: 04, Epoch: 45, Loss: 1.2219, Train: 57.47%, Valid: 54.24% Test: 64.86%
Run: 04, Epoch: 46, Loss: 1.1670, Train: 59.77%, Valid: 55.93% Test: 64.86%
Run: 04, Epoch: 47, Loss: 1.1577, Train: 59.77%, Valid: 55.93% Test: 64.86%
Run: 04, Epoch: 48, Loss: 1.1940, Train: 60.92%, Valid: 55.93% Test: 64.86%
Run: 04, Epoch: 49, Loss: 1.0604, Train: 59.77%, Valid: 55.93% Test: 64.86%
Run: 04, Epo

Run: 08, Epoch: 37, Loss: 1.2977, Train: 58.62%, Valid: 64.41% Test: 67.57%
Run: 08, Epoch: 38, Loss: 1.1507, Train: 58.62%, Valid: 64.41% Test: 67.57%
Run: 08, Epoch: 39, Loss: 1.1852, Train: 59.77%, Valid: 62.71% Test: 67.57%
Run: 08, Epoch: 40, Loss: 1.1402, Train: 60.92%, Valid: 64.41% Test: 67.57%
Run: 08, Epoch: 41, Loss: 1.1498, Train: 57.47%, Valid: 64.41% Test: 67.57%
Run: 08, Epoch: 42, Loss: 1.1416, Train: 57.47%, Valid: 64.41% Test: 67.57%
Run: 08, Epoch: 43, Loss: 1.2720, Train: 57.47%, Valid: 64.41% Test: 64.86%
Run: 08, Epoch: 44, Loss: 1.1392, Train: 57.47%, Valid: 62.71% Test: 67.57%
Run: 08, Epoch: 45, Loss: 1.1899, Train: 57.47%, Valid: 66.10% Test: 67.57%
Run: 08, Epoch: 46, Loss: 1.2137, Train: 57.47%, Valid: 66.10% Test: 67.57%
Run: 08, Epoch: 47, Loss: 1.2091, Train: 58.62%, Valid: 66.10% Test: 67.57%
Run: 08, Epoch: 48, Loss: 1.1336, Train: 59.77%, Valid: 66.10% Test: 67.57%
Run: 08, Epoch: 49, Loss: 1.0346, Train: 60.92%, Valid: 66.10% Test: 67.57%
Run: 08, Epo

# Topological Encoding 

In [35]:
dataset = WebKB(root='/tmp/Texas', name='Texas')
data = dataset[0]
print(data)

Data(x=[183, 1703], edge_index=[2, 325], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10])


In [37]:
Edge_idx=data.edge_index.numpy()
Node=range(Number_nodes)
Edgelist=[]
for i in range(len(Edge_idx[1])):
    Edgelist.append((Edge_idx[0][i],Edge_idx[1][i]))
#print(Edgelist)

In [38]:
# a "plain" graph is undirected
G = nx.DiGraph()

# give each a node a 'name', which is a letter in this case.
#G.add_node('a')

# the add_nodes_from method allows adding nodes from a sequence, in this case a list
#nodes_to_add = ['b', 'c', 'd']
G.add_nodes_from(Node)

# add edge from 'a' to 'b'
# since this graph is undirected, the order doesn't matter here
#G.add_edge('a', 'b')

# just like add_nodes_from, we can add edges from a sequence
# edges should be specified as 2-tuples
#edges_to_add = [('a', 'c'), ('b', 'c'), ('c', 'd')]
G.add_edges_from(Edgelist)


In [ ]:
print(G.number_of_edges())

In [39]:
def Topological_Feature_subLevel(adj,filtration_fun, Filtration):
        betti_0=[]
        betti_1=[]
        for p in range(len(Filtration)):
            n_active = np.where(np.array(filtration_fun) <= Filtration[p])[0].tolist()
            Active_node=np.unique(n_active)
            if (len(Active_node)==0):
                betti_0.append(0)
                betti_1.append(0)
            else:
                b=adj[Active_node,:][:,Active_node]
                my_flag=pyflagser.flagser_unweighted(b, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                x = my_flag["betti"]
                betti_0.append(x[0])
                betti_1.append(x[1])
            n_active.clear()
        return betti_0,betti_1

In [40]:
def Degree_list(Graph):
    degree_list = [Graph.degree(node) for node in Graph.nodes]
    return np.array(degree_list)

In [41]:
degree_list=Degree_list(G)
unique_list=np.unique(degree_list)
for d in unique_list:
    count=0
    for i in range(len(degree_list)):
        if degree_list[i]==d:
            count=count+1
    print(int(d)," | ",count,'\n')

1  |  60 

2  |  42 

3  |  33 

4  |  19 

5  |  7 

6  |  2 

7  |  4 

8  |  5 

9  |  4 

10  |  2 

11  |  1 

12  |  1 

13  |  1 

20  |  1 

104  |  1 



In [42]:
import pyflagser
Node_fil=[1,2,3,4,5,6,7,8,9,10,20,100]
topo_betti_0=[]
topo_betti_1=[]
Node_Edge=[]
for i in range(Number_nodes):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(Number_nodes-1)), end='', flush=True)
    subgraph=ego_graph(G, i, radius=2, center=True, undirected=True, distance=None)
    filt=Degree_list(subgraph)
    A_sub = nx.to_numpy_array(subgraph)# adjacency matrix of subgraph
    fe=Topological_Feature_subLevel(A_sub,filt,Node_fil)
    topo_betti_0.append(fe[0])
    topo_betti_1.append(fe[1])
    Node_Edge.append([subgraph.number_of_nodes(),subgraph.number_of_edges()])
    #topo_with_NE.app

Processing file 182 (100%)

In [ ]:
print( Node_Edge)

In [43]:
dataset = WebKB(root='/tmp/Texas', name='Texas',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[183, 1703], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=325])


In [44]:
topo_betti0=torch.tensor(topo_betti_0).float()
topo_betti1=torch.tensor(topo_betti_1).float()
NodeEdge=torch.tensor(Node_Edge).float()

In [45]:
data.x=CC_domain
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[183, 10], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=325], topo=[183, 24])


# TOPO-W_GAT

In [46]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout,heads):
        super(GAT, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(GATConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GATConv(hidden_channels, out_channels))

        self.dropout = dropout
        self.heads=heads

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x
        #return x.log_softmax(dim=-1)

class MLP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        #return torch.log_softmax(x, dim=-1)
        return x
    
class MLP2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP2, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp2(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.log_softmax(x, dim=-1)
    

def train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2):
    model.train()
    mlp_model.train()
    mlp_2.train()
    optimizer.zero_grad()
    optimizer_mlp.zero_grad()
    optimizer_mlp2.zero_grad()
    gcn_embedding = model(data.x, data.adj_t)[train_idx]
    #print(gcn_embedding)
    mlp_embedding = mlp_model(data.topo[train_idx])
    #print(mlp_embedding)
    combined_embedding = torch.cat((gcn_embedding, mlp_embedding), dim=1)
    #print(combined_embedding)
    mlp_emb = mlp_2(combined_embedding)
    #print(mlp_emb)
    loss = F.nll_loss(mlp_emb, data.y.squeeze()[train_idx])
    #loss = F.nll_loss(combined_embedding, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer_mlp2.step()
    optimizer.step()
    optimizer_mlp.step()
    

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total



@torch.no_grad()
def test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx):
    model.eval()
    mlp_model.eval()
    mlp_2.eval()

    gcn_out = model(data.x, data.adj_t)
    #print(gcn_out[0])
    mlp_out=mlp_model(data.topo)
    #print(mlp_out)
    #out=torch.cat((gcn_out,mlp_out),dim=1)
    Com=torch.cat((gcn_out,mlp_out),dim=1)
    out=mlp_2(Com)
    y_pred = out.argmax(dim=-1, keepdim=True)
    #print(y_pred[0])
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [52]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 8, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}
    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    model = GAT(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout,args.heads)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 10,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(20, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.001)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.5385, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 02, Loss: 1.4893, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 03, Loss: 1.4220, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 04, Loss: 1.3782, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 05, Loss: 1.3485, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 06, Loss: 1.2993, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 07, Loss: 1.3212, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 08, Loss: 1.2214, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 09, Loss: 1.1907, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 10, Loss: 1.2036, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 11, Loss: 1.1286, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 12, Loss: 1.1988, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epoch: 13, Loss: 1.1456, Train: 52.87%, Valid: 52.54% Test: 64.86%
Run: 01, Epo

Run: 01, Epoch: 137, Loss: 0.8694, Train: 67.82%, Valid: 64.41% Test: 59.46%
Run: 01, Epoch: 138, Loss: 0.9109, Train: 67.82%, Valid: 64.41% Test: 59.46%
Run: 01, Epoch: 139, Loss: 0.8686, Train: 64.37%, Valid: 67.80% Test: 70.27%
Run: 01, Epoch: 140, Loss: 0.7882, Train: 67.82%, Valid: 66.10% Test: 64.86%
Run: 01, Epoch: 141, Loss: 0.8268, Train: 65.52%, Valid: 66.10% Test: 64.86%
Run: 01, Epoch: 142, Loss: 0.8554, Train: 66.67%, Valid: 64.41% Test: 62.16%
Run: 01, Epoch: 143, Loss: 0.9356, Train: 68.97%, Valid: 64.41% Test: 62.16%
Run: 01, Epoch: 144, Loss: 0.9478, Train: 67.82%, Valid: 64.41% Test: 64.86%
Run: 01, Epoch: 145, Loss: 0.8516, Train: 68.97%, Valid: 64.41% Test: 64.86%
Run: 01, Epoch: 146, Loss: 0.8742, Train: 70.11%, Valid: 64.41% Test: 64.86%
Run: 01, Epoch: 147, Loss: 0.8817, Train: 70.11%, Valid: 64.41% Test: 64.86%
Run: 01, Epoch: 148, Loss: 0.9648, Train: 71.26%, Valid: 62.71% Test: 59.46%
Run: 01, Epoch: 149, Loss: 0.8975, Train: 73.56%, Valid: 61.02% Test: 59.46%

Run: 02, Epoch: 72, Loss: 0.9676, Train: 63.22%, Valid: 57.63% Test: 70.27%
Run: 02, Epoch: 73, Loss: 0.8715, Train: 63.22%, Valid: 57.63% Test: 70.27%
Run: 02, Epoch: 74, Loss: 0.9263, Train: 63.22%, Valid: 57.63% Test: 70.27%
Run: 02, Epoch: 75, Loss: 0.8617, Train: 63.22%, Valid: 57.63% Test: 70.27%
Run: 02, Epoch: 76, Loss: 0.9213, Train: 63.22%, Valid: 57.63% Test: 70.27%
Run: 02, Epoch: 77, Loss: 0.9578, Train: 63.22%, Valid: 59.32% Test: 70.27%
Run: 02, Epoch: 78, Loss: 0.9187, Train: 63.22%, Valid: 59.32% Test: 70.27%
Run: 02, Epoch: 79, Loss: 0.9636, Train: 63.22%, Valid: 59.32% Test: 70.27%
Run: 02, Epoch: 80, Loss: 0.9177, Train: 63.22%, Valid: 59.32% Test: 70.27%
Run: 02, Epoch: 81, Loss: 0.9279, Train: 63.22%, Valid: 59.32% Test: 70.27%
Run: 02, Epoch: 82, Loss: 0.9462, Train: 63.22%, Valid: 59.32% Test: 67.57%
Run: 02, Epoch: 83, Loss: 0.9156, Train: 65.52%, Valid: 59.32% Test: 67.57%
Run: 02, Epoch: 84, Loss: 0.9436, Train: 65.52%, Valid: 61.02% Test: 72.97%
Run: 02, Epo

Run: 03, Epoch: 11, Loss: 1.2074, Train: 58.62%, Valid: 54.24% Test: 48.65%
Run: 03, Epoch: 12, Loss: 1.2176, Train: 58.62%, Valid: 54.24% Test: 48.65%
Run: 03, Epoch: 13, Loss: 1.1889, Train: 58.62%, Valid: 54.24% Test: 48.65%
Run: 03, Epoch: 14, Loss: 1.1276, Train: 58.62%, Valid: 54.24% Test: 48.65%
Run: 03, Epoch: 15, Loss: 1.1215, Train: 58.62%, Valid: 54.24% Test: 48.65%
Run: 03, Epoch: 16, Loss: 1.1074, Train: 58.62%, Valid: 54.24% Test: 48.65%
Run: 03, Epoch: 17, Loss: 1.1156, Train: 58.62%, Valid: 54.24% Test: 48.65%
Run: 03, Epoch: 18, Loss: 1.1120, Train: 58.62%, Valid: 54.24% Test: 48.65%
Run: 03, Epoch: 19, Loss: 1.0897, Train: 58.62%, Valid: 54.24% Test: 48.65%
Run: 03, Epoch: 20, Loss: 1.0744, Train: 58.62%, Valid: 54.24% Test: 48.65%
Run: 03, Epoch: 21, Loss: 1.0672, Train: 58.62%, Valid: 54.24% Test: 48.65%
Run: 03, Epoch: 22, Loss: 1.0004, Train: 58.62%, Valid: 54.24% Test: 48.65%
Run: 03, Epoch: 23, Loss: 1.0249, Train: 58.62%, Valid: 54.24% Test: 48.65%
Run: 03, Epo

Run: 03, Epoch: 155, Loss: 0.7185, Train: 71.26%, Valid: 62.71% Test: 45.95%
Run: 03, Epoch: 156, Loss: 0.6718, Train: 72.41%, Valid: 64.41% Test: 45.95%
Run: 03, Epoch: 157, Loss: 0.6991, Train: 73.56%, Valid: 62.71% Test: 48.65%
Run: 03, Epoch: 158, Loss: 0.6744, Train: 77.01%, Valid: 61.02% Test: 48.65%
Run: 03, Epoch: 159, Loss: 0.6491, Train: 77.01%, Valid: 64.41% Test: 51.35%
Run: 03, Epoch: 160, Loss: 0.6867, Train: 77.01%, Valid: 62.71% Test: 51.35%
Run: 03, Epoch: 161, Loss: 0.6257, Train: 75.86%, Valid: 66.10% Test: 54.05%
Run: 03, Epoch: 162, Loss: 0.6123, Train: 77.01%, Valid: 64.41% Test: 54.05%
Run: 03, Epoch: 163, Loss: 0.6452, Train: 75.86%, Valid: 64.41% Test: 54.05%
Run: 03, Epoch: 164, Loss: 0.6219, Train: 74.71%, Valid: 62.71% Test: 51.35%
Run: 03, Epoch: 165, Loss: 0.6874, Train: 77.01%, Valid: 64.41% Test: 48.65%
Run: 03, Epoch: 166, Loss: 0.6921, Train: 77.01%, Valid: 64.41% Test: 48.65%
Run: 03, Epoch: 167, Loss: 0.6688, Train: 75.86%, Valid: 64.41% Test: 48.65%

...

Run: 07, Epoch: 193, Loss: 0.7555, Train: 74.71%, Valid: 62.71% Test: 62.16%
Run: 07, Epoch: 194, Loss: 0.7897, Train: 73.56%, Valid: 61.02% Test: 62.16%
Run: 07, Epoch: 195, Loss: 0.7516, Train: 72.41%, Valid: 61.02% Test: 62.16%
Run: 07, Epoch: 196, Loss: 0.7683, Train: 72.41%, Valid: 61.02% Test: 62.16%
Run: 07, Epoch: 197, Loss: 0.8229, Train: 71.26%, Valid: 61.02% Test: 62.16%
Run: 07, Epoch: 198, Loss: 0.8156, Train: 75.86%, Valid: 62.71% Test: 62.16%
Run: 07, Epoch: 199, Loss: 0.7618, Train: 75.86%, Valid: 66.10% Test: 70.27%
Run: 07, Epoch: 200, Loss: 0.7796, Train: 73.56%, Valid: 66.10% Test: 70.27%
Run 07:
Highest Train: 75.86
Highest Valid: 71.19
  Final Train: 66.67
   Final Test: 62.16
Run: 08, Epoch: 01, Loss: 1.5357, Train: 47.13%, Valid: 62.71% Test: 62.16%
Run: 08, Epoch: 02, Loss: 1.4719, Train: 47.13%, Valid: 62.71% Test: 62.16%
Run: 08, Epoch: 03, Loss: 1.4171, Train: 47.13%, Valid: 62.71% Test: 62.16%
Run: 08, Epoch: 04, Loss: 1.4144, Train: 47.13%, Valid: 62.71% T

Run: 08, Epoch: 133, Loss: 0.8368, Train: 63.22%, Valid: 52.54% Test: 59.46%
Run: 08, Epoch: 134, Loss: 0.7799, Train: 66.67%, Valid: 57.63% Test: 62.16%
Run: 08, Epoch: 135, Loss: 0.7988, Train: 67.82%, Valid: 61.02% Test: 67.57%
Run: 08, Epoch: 136, Loss: 0.7927, Train: 67.82%, Valid: 61.02% Test: 70.27%
Run: 08, Epoch: 137, Loss: 0.8550, Train: 66.67%, Valid: 61.02% Test: 70.27%
Run: 08, Epoch: 138, Loss: 0.7878, Train: 65.52%, Valid: 61.02% Test: 70.27%
Run: 08, Epoch: 139, Loss: 0.7896, Train: 65.52%, Valid: 61.02% Test: 70.27%
Run: 08, Epoch: 140, Loss: 0.8214, Train: 68.97%, Valid: 61.02% Test: 70.27%
Run: 08, Epoch: 141, Loss: 0.8833, Train: 68.97%, Valid: 62.71% Test: 70.27%
Run: 08, Epoch: 142, Loss: 0.7852, Train: 71.26%, Valid: 62.71% Test: 70.27%
Run: 08, Epoch: 143, Loss: 0.7783, Train: 73.56%, Valid: 62.71% Test: 70.27%
Run: 08, Epoch: 144, Loss: 0.7795, Train: 75.86%, Valid: 62.71% Test: 70.27%
Run: 08, Epoch: 145, Loss: 0.8213, Train: 74.71%, Valid: 61.02% Test: 70.27%

Run: 09, Epoch: 75, Loss: 0.9652, Train: 65.52%, Valid: 44.07% Test: 64.86%
Run: 09, Epoch: 76, Loss: 0.9137, Train: 64.37%, Valid: 44.07% Test: 64.86%
Run: 09, Epoch: 77, Loss: 0.9483, Train: 64.37%, Valid: 45.76% Test: 64.86%
Run: 09, Epoch: 78, Loss: 0.9149, Train: 62.07%, Valid: 42.37% Test: 62.16%
Run: 09, Epoch: 79, Loss: 0.9502, Train: 62.07%, Valid: 42.37% Test: 62.16%
Run: 09, Epoch: 80, Loss: 0.8981, Train: 62.07%, Valid: 42.37% Test: 62.16%
Run: 09, Epoch: 81, Loss: 0.9204, Train: 63.22%, Valid: 45.76% Test: 64.86%
Run: 09, Epoch: 82, Loss: 0.9149, Train: 63.22%, Valid: 45.76% Test: 64.86%
Run: 09, Epoch: 83, Loss: 0.9132, Train: 63.22%, Valid: 45.76% Test: 64.86%
Run: 09, Epoch: 84, Loss: 0.9536, Train: 67.82%, Valid: 57.63% Test: 70.27%
Run: 09, Epoch: 85, Loss: 0.9070, Train: 64.37%, Valid: 55.93% Test: 70.27%
Run: 09, Epoch: 86, Loss: 0.8565, Train: 64.37%, Valid: 55.93% Test: 70.27%
Run: 09, Epoch: 87, Loss: 0.8839, Train: 64.37%, Valid: 55.93% Test: 70.27%
Run: 09, Epo

Run: 10, Epoch: 15, Loss: 1.2259, Train: 64.37%, Valid: 50.85% Test: 67.57%
Run: 10, Epoch: 16, Loss: 1.2128, Train: 63.22%, Valid: 52.54% Test: 67.57%
Run: 10, Epoch: 17, Loss: 1.2507, Train: 63.22%, Valid: 52.54% Test: 67.57%
Run: 10, Epoch: 18, Loss: 1.1909, Train: 63.22%, Valid: 52.54% Test: 67.57%
Run: 10, Epoch: 19, Loss: 1.1744, Train: 63.22%, Valid: 52.54% Test: 67.57%
Run: 10, Epoch: 20, Loss: 1.1773, Train: 63.22%, Valid: 52.54% Test: 67.57%
Run: 10, Epoch: 21, Loss: 1.0880, Train: 63.22%, Valid: 52.54% Test: 67.57%
Run: 10, Epoch: 22, Loss: 1.1073, Train: 63.22%, Valid: 52.54% Test: 67.57%
Run: 10, Epoch: 23, Loss: 1.1149, Train: 63.22%, Valid: 54.24% Test: 67.57%
Run: 10, Epoch: 24, Loss: 1.1130, Train: 63.22%, Valid: 54.24% Test: 67.57%
Run: 10, Epoch: 25, Loss: 1.0599, Train: 63.22%, Valid: 54.24% Test: 67.57%
Run: 10, Epoch: 26, Loss: 1.0977, Train: 63.22%, Valid: 54.24% Test: 67.57%
Run: 10, Epoch: 27, Loss: 1.0829, Train: 63.22%, Valid: 54.24% Test: 67.57%
Run: 10, Epo

Run: 10, Epoch: 155, Loss: 0.7969, Train: 66.67%, Valid: 54.24% Test: 72.97%
Run: 10, Epoch: 156, Loss: 0.9057, Train: 67.82%, Valid: 54.24% Test: 72.97%
Run: 10, Epoch: 157, Loss: 0.8020, Train: 66.67%, Valid: 54.24% Test: 72.97%
Run: 10, Epoch: 158, Loss: 0.8374, Train: 62.07%, Valid: 57.63% Test: 70.27%
Run: 10, Epoch: 159, Loss: 0.7900, Train: 63.22%, Valid: 57.63% Test: 67.57%
Run: 10, Epoch: 160, Loss: 0.8635, Train: 63.22%, Valid: 57.63% Test: 67.57%
Run: 10, Epoch: 161, Loss: 0.7317, Train: 67.82%, Valid: 57.63% Test: 70.27%
Run: 10, Epoch: 162, Loss: 0.7855, Train: 67.82%, Valid: 52.54% Test: 70.27%
Run: 10, Epoch: 163, Loss: 0.9207, Train: 67.82%, Valid: 59.32% Test: 72.97%
Run: 10, Epoch: 164, Loss: 0.8190, Train: 68.97%, Valid: 59.32% Test: 72.97%
Run: 10, Epoch: 165, Loss: 0.7579, Train: 70.11%, Valid: 59.32% Test: 72.97%
Run: 10, Epoch: 166, Loss: 0.8830, Train: 68.97%, Valid: 59.32% Test: 72.97%
Run: 10, Epoch: 167, Loss: 0.8665, Train: 68.97%, Valid: 59.32% Test: 72.97%

# TOPO_GAT

In [53]:
dataset = WebKB(root='/tmp/Texas', name='Texas',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[183, 1703], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=325])


In [54]:
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[183, 1703], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=325], topo=[183, 24])


In [58]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 8, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}
    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    model = GAT(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout,args.heads)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 10,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(20, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.001)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.7170, Train: 16.09%, Valid: 25.42% Test: 10.81%
Run: 01, Epoch: 02, Loss: 1.6392, Train: 16.09%, Valid: 25.42% Test: 10.81%
Run: 01, Epoch: 03, Loss: 1.5740, Train: 16.09%, Valid: 25.42% Test: 10.81%
Run: 01, Epoch: 04, Loss: 1.4845, Train: 16.09%, Valid: 25.42% Test: 10.81%
Run: 01, Epoch: 05, Loss: 1.4288, Train: 16.09%, Valid: 25.42% Test: 10.81%
Run: 01, Epoch: 06, Loss: 1.3525, Train: 16.09%, Valid: 25.42% Test: 10.81%
Run: 01, Epoch: 07, Loss: 1.2665, Train: 16.09%, Valid: 25.42% Test: 10.81%
Run: 01, Epoch: 08, Loss: 1.2116, Train: 16.09%, Valid: 25.42% Test: 10.81%
Run: 01, Epoch: 09, Loss: 1.1632, Train: 43.68%, Valid: 42.37% Test: 18.92%
Run: 01, Epoch: 10, Loss: 1.0839, Train: 78.16%, Valid: 55.93% Test: 51.35%
Run: 01, Epoch: 11, Loss: 1.0408, Train: 83.91%, Valid: 49.15% Test: 51.35%
Run: 01, Epoch: 12, Loss: 1.0141, Train: 81.61%, Valid: 49.15% Test: 59.46%
Run: 01, Epoch: 13, Loss: 0.9560, Train: 80.46%, Valid: 50.85% Test: 62.16%
Run: 01, Epo

Run: 01, Epoch: 130, Loss: 0.2639, Train: 100.00%, Valid: 44.07% Test: 35.14%
Run: 01, Epoch: 131, Loss: 0.3584, Train: 100.00%, Valid: 45.76% Test: 35.14%
Run: 01, Epoch: 132, Loss: 0.2871, Train: 100.00%, Valid: 47.46% Test: 35.14%
Run: 01, Epoch: 133, Loss: 0.2874, Train: 98.85%, Valid: 45.76% Test: 32.43%
Run: 01, Epoch: 134, Loss: 0.2586, Train: 98.85%, Valid: 45.76% Test: 29.73%
Run: 01, Epoch: 135, Loss: 0.2896, Train: 98.85%, Valid: 44.07% Test: 29.73%
Run: 01, Epoch: 136, Loss: 0.3131, Train: 98.85%, Valid: 44.07% Test: 32.43%
Run: 01, Epoch: 137, Loss: 0.2094, Train: 98.85%, Valid: 44.07% Test: 32.43%
Run: 01, Epoch: 138, Loss: 0.2830, Train: 97.70%, Valid: 45.76% Test: 32.43%
Run: 01, Epoch: 139, Loss: 0.2708, Train: 96.55%, Valid: 47.46% Test: 35.14%
Run: 01, Epoch: 140, Loss: 0.2400, Train: 96.55%, Valid: 47.46% Test: 35.14%
Run: 01, Epoch: 141, Loss: 0.2260, Train: 94.25%, Valid: 47.46% Test: 35.14%
Run: 01, Epoch: 142, Loss: 0.2125, Train: 94.25%, Valid: 44.07% Test: 35.

Run: 02, Epoch: 62, Loss: 0.6062, Train: 83.91%, Valid: 42.37% Test: 54.05%
Run: 02, Epoch: 63, Loss: 0.6040, Train: 83.91%, Valid: 44.07% Test: 54.05%
Run: 02, Epoch: 64, Loss: 0.5887, Train: 85.06%, Valid: 44.07% Test: 54.05%
Run: 02, Epoch: 65, Loss: 0.5794, Train: 85.06%, Valid: 44.07% Test: 54.05%
Run: 02, Epoch: 66, Loss: 0.5855, Train: 85.06%, Valid: 47.46% Test: 54.05%
Run: 02, Epoch: 67, Loss: 0.5442, Train: 83.91%, Valid: 47.46% Test: 54.05%
Run: 02, Epoch: 68, Loss: 0.5739, Train: 87.36%, Valid: 47.46% Test: 59.46%
Run: 02, Epoch: 69, Loss: 0.5330, Train: 87.36%, Valid: 45.76% Test: 59.46%
Run: 02, Epoch: 70, Loss: 0.5849, Train: 87.36%, Valid: 44.07% Test: 56.76%
Run: 02, Epoch: 71, Loss: 0.5491, Train: 87.36%, Valid: 42.37% Test: 56.76%
Run: 02, Epoch: 72, Loss: 0.5489, Train: 87.36%, Valid: 42.37% Test: 56.76%
Run: 02, Epoch: 73, Loss: 0.5711, Train: 88.51%, Valid: 40.68% Test: 56.76%
Run: 02, Epoch: 74, Loss: 0.5582, Train: 89.66%, Valid: 42.37% Test: 56.76%
Run: 02, Epo

Run: 02, Epoch: 193, Loss: 0.3308, Train: 96.55%, Valid: 38.98% Test: 59.46%
Run: 02, Epoch: 194, Loss: 0.3158, Train: 95.40%, Valid: 38.98% Test: 59.46%
Run: 02, Epoch: 195, Loss: 0.2522, Train: 95.40%, Valid: 38.98% Test: 59.46%
Run: 02, Epoch: 196, Loss: 0.3480, Train: 95.40%, Valid: 38.98% Test: 59.46%
Run: 02, Epoch: 197, Loss: 0.3270, Train: 95.40%, Valid: 38.98% Test: 59.46%
Run: 02, Epoch: 198, Loss: 0.3637, Train: 95.40%, Valid: 38.98% Test: 59.46%
Run: 02, Epoch: 199, Loss: 0.2653, Train: 95.40%, Valid: 38.98% Test: 62.16%
Run: 02, Epoch: 200, Loss: 0.2912, Train: 94.25%, Valid: 38.98% Test: 62.16%
Run 02:
Highest Train: 96.55
Highest Valid: 54.24
  Final Train: 56.32
   Final Test: 62.16
Run: 03, Epoch: 01, Loss: 1.6081, Train: 0.00%, Valid: 0.00% Test: 2.70%
Run: 03, Epoch: 02, Loss: 1.5186, Train: 58.62%, Valid: 54.24% Test: 48.65%
Run: 03, Epoch: 03, Loss: 1.4498, Train: 58.62%, Valid: 54.24% Test: 48.65%
Run: 03, Epoch: 04, Loss: 1.3795, Train: 58.62%, Valid: 54.24% Test

Run: 03, Epoch: 127, Loss: 0.2754, Train: 98.85%, Valid: 42.37% Test: 35.14%
Run: 03, Epoch: 128, Loss: 0.2635, Train: 98.85%, Valid: 40.68% Test: 35.14%
Run: 03, Epoch: 129, Loss: 0.2260, Train: 98.85%, Valid: 42.37% Test: 32.43%
Run: 03, Epoch: 130, Loss: 0.2379, Train: 98.85%, Valid: 40.68% Test: 29.73%
Run: 03, Epoch: 131, Loss: 0.2360, Train: 98.85%, Valid: 40.68% Test: 29.73%
Run: 03, Epoch: 132, Loss: 0.2090, Train: 98.85%, Valid: 40.68% Test: 32.43%
Run: 03, Epoch: 133, Loss: 0.2219, Train: 98.85%, Valid: 40.68% Test: 35.14%
Run: 03, Epoch: 134, Loss: 0.2831, Train: 98.85%, Valid: 40.68% Test: 35.14%
Run: 03, Epoch: 135, Loss: 0.1973, Train: 98.85%, Valid: 40.68% Test: 35.14%
Run: 03, Epoch: 136, Loss: 0.2143, Train: 98.85%, Valid: 40.68% Test: 37.84%
Run: 03, Epoch: 137, Loss: 0.2114, Train: 98.85%, Valid: 40.68% Test: 37.84%
Run: 03, Epoch: 138, Loss: 0.2123, Train: 98.85%, Valid: 40.68% Test: 40.54%
Run: 03, Epoch: 139, Loss: 0.2466, Train: 98.85%, Valid: 42.37% Test: 37.84%

...

Run: 08, Epoch: 49, Loss: 0.5532, Train: 89.66%, Valid: 59.32% Test: 48.65%
Run: 08, Epoch: 50, Loss: 0.4972, Train: 90.80%, Valid: 59.32% Test: 48.65%
Run: 08, Epoch: 51, Loss: 0.4984, Train: 90.80%, Valid: 59.32% Test: 51.35%
Run: 08, Epoch: 52, Loss: 0.5164, Train: 90.80%, Valid: 61.02% Test: 51.35%
Run: 08, Epoch: 53, Loss: 0.4834, Train: 91.95%, Valid: 61.02% Test: 51.35%
Run: 08, Epoch: 54, Loss: 0.5305, Train: 91.95%, Valid: 61.02% Test: 54.05%
Run: 08, Epoch: 55, Loss: 0.5261, Train: 90.80%, Valid: 62.71% Test: 51.35%
Run: 08, Epoch: 56, Loss: 0.4970, Train: 89.66%, Valid: 64.41% Test: 51.35%
Run: 08, Epoch: 57, Loss: 0.4803, Train: 89.66%, Valid: 61.02% Test: 51.35%
Run: 08, Epoch: 58, Loss: 0.4718, Train: 89.66%, Valid: 61.02% Test: 51.35%
Run: 08, Epoch: 59, Loss: 0.4671, Train: 88.51%, Valid: 62.71% Test: 51.35%
Run: 08, Epoch: 60, Loss: 0.4446, Train: 88.51%, Valid: 62.71% Test: 51.35%
Run: 08, Epoch: 61, Loss: 0.5256, Train: 89.66%, Valid: 62.71% Test: 51.35%
Run: 08, Epo

Run: 08, Epoch: 179, Loss: 0.1736, Train: 98.85%, Valid: 52.54% Test: 43.24%
Run: 08, Epoch: 180, Loss: 0.1757, Train: 98.85%, Valid: 52.54% Test: 43.24%
Run: 08, Epoch: 181, Loss: 0.1967, Train: 100.00%, Valid: 52.54% Test: 43.24%
Run: 08, Epoch: 182, Loss: 0.1443, Train: 100.00%, Valid: 50.85% Test: 43.24%
Run: 08, Epoch: 183, Loss: 0.1823, Train: 100.00%, Valid: 49.15% Test: 43.24%
Run: 08, Epoch: 184, Loss: 0.1663, Train: 100.00%, Valid: 49.15% Test: 43.24%
Run: 08, Epoch: 185, Loss: 0.1661, Train: 100.00%, Valid: 49.15% Test: 45.95%
Run: 08, Epoch: 186, Loss: 0.2283, Train: 100.00%, Valid: 50.85% Test: 45.95%
Run: 08, Epoch: 187, Loss: 0.2394, Train: 100.00%, Valid: 52.54% Test: 45.95%
Run: 08, Epoch: 188, Loss: 0.1657, Train: 100.00%, Valid: 52.54% Test: 45.95%
Run: 08, Epoch: 189, Loss: 0.1407, Train: 100.00%, Valid: 50.85% Test: 45.95%
Run: 08, Epoch: 190, Loss: 0.1710, Train: 100.00%, Valid: 50.85% Test: 48.65%
Run: 08, Epoch: 191, Loss: 0.1554, Train: 100.00%, Valid: 54.24% T

Run: 09, Epoch: 110, Loss: 0.3165, Train: 94.25%, Valid: 35.59% Test: 45.95%
Run: 09, Epoch: 111, Loss: 0.3069, Train: 94.25%, Valid: 33.90% Test: 45.95%
Run: 09, Epoch: 112, Loss: 0.3154, Train: 94.25%, Valid: 32.20% Test: 45.95%
Run: 09, Epoch: 113, Loss: 0.2924, Train: 94.25%, Valid: 33.90% Test: 45.95%
Run: 09, Epoch: 114, Loss: 0.2909, Train: 94.25%, Valid: 35.59% Test: 45.95%
Run: 09, Epoch: 115, Loss: 0.2756, Train: 94.25%, Valid: 33.90% Test: 43.24%
Run: 09, Epoch: 116, Loss: 0.2876, Train: 94.25%, Valid: 30.51% Test: 40.54%
Run: 09, Epoch: 117, Loss: 0.3082, Train: 94.25%, Valid: 30.51% Test: 40.54%
Run: 09, Epoch: 118, Loss: 0.3558, Train: 95.40%, Valid: 30.51% Test: 40.54%
Run: 09, Epoch: 119, Loss: 0.3146, Train: 95.40%, Valid: 35.59% Test: 43.24%
Run: 09, Epoch: 120, Loss: 0.3038, Train: 95.40%, Valid: 35.59% Test: 43.24%
Run: 09, Epoch: 121, Loss: 0.2845, Train: 95.40%, Valid: 35.59% Test: 43.24%
Run: 09, Epoch: 122, Loss: 0.2993, Train: 95.40%, Valid: 35.59% Test: 43.24%

Run: 10, Epoch: 39, Loss: 0.6108, Train: 86.21%, Valid: 54.24% Test: 67.57%
Run: 10, Epoch: 40, Loss: 0.6421, Train: 87.36%, Valid: 47.46% Test: 72.97%
Run: 10, Epoch: 41, Loss: 0.6189, Train: 87.36%, Valid: 49.15% Test: 70.27%
Run: 10, Epoch: 42, Loss: 0.5965, Train: 86.21%, Valid: 54.24% Test: 67.57%
Run: 10, Epoch: 43, Loss: 0.5990, Train: 85.06%, Valid: 54.24% Test: 64.86%
Run: 10, Epoch: 44, Loss: 0.6076, Train: 83.91%, Valid: 55.93% Test: 64.86%
Run: 10, Epoch: 45, Loss: 0.5620, Train: 83.91%, Valid: 52.54% Test: 64.86%
Run: 10, Epoch: 46, Loss: 0.5948, Train: 83.91%, Valid: 52.54% Test: 64.86%
Run: 10, Epoch: 47, Loss: 0.6412, Train: 87.36%, Valid: 50.85% Test: 64.86%
Run: 10, Epoch: 48, Loss: 0.6005, Train: 88.51%, Valid: 49.15% Test: 67.57%
Run: 10, Epoch: 49, Loss: 0.5635, Train: 88.51%, Valid: 49.15% Test: 67.57%
Run: 10, Epoch: 50, Loss: 0.5462, Train: 87.36%, Valid: 45.76% Test: 67.57%
Run: 10, Epoch: 51, Loss: 0.5621, Train: 83.91%, Valid: 45.76% Test: 64.86%
Run: 10, Epo

Run: 10, Epoch: 171, Loss: 0.2503, Train: 97.70%, Valid: 40.68% Test: 59.46%
Run: 10, Epoch: 172, Loss: 0.2586, Train: 97.70%, Valid: 40.68% Test: 59.46%
Run: 10, Epoch: 173, Loss: 0.2790, Train: 97.70%, Valid: 40.68% Test: 59.46%
Run: 10, Epoch: 174, Loss: 0.3217, Train: 97.70%, Valid: 38.98% Test: 59.46%
Run: 10, Epoch: 175, Loss: 0.2738, Train: 97.70%, Valid: 38.98% Test: 56.76%
Run: 10, Epoch: 176, Loss: 0.2279, Train: 97.70%, Valid: 38.98% Test: 59.46%
Run: 10, Epoch: 177, Loss: 0.2175, Train: 97.70%, Valid: 38.98% Test: 59.46%
Run: 10, Epoch: 178, Loss: 0.2471, Train: 97.70%, Valid: 38.98% Test: 59.46%
Run: 10, Epoch: 179, Loss: 0.2567, Train: 97.70%, Valid: 38.98% Test: 56.76%
Run: 10, Epoch: 180, Loss: 0.2565, Train: 97.70%, Valid: 38.98% Test: 56.76%
Run: 10, Epoch: 181, Loss: 0.2574, Train: 97.70%, Valid: 38.98% Test: 56.76%
Run: 10, Epoch: 182, Loss: 0.1898, Train: 97.70%, Valid: 38.98% Test: 56.76%
Run: 10, Epoch: 183, Loss: 0.2960, Train: 97.70%, Valid: 40.68% Test: 56.76%